In [144]:
#The Black-Scholes function
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import scipy.stats as ss


#Black and Scholes model
def d1(S0, K, r, sigma, T):
    return (np.log(S0/K) + (r + sigma**2 / 2) * T)/(sigma * np.sqrt(T))
 
    
def d2(S0, K, r, sigma, T):
    return (np.log(S0/K) + (r - sigma**2 / 2) * T)/(sigma * np.sqrt(T))
 
    
def BlackScholes(S0, K, r, sigma, T):
    return S0*ss.norm.cdf(d1(S0, K, r, sigma, T)) - \
           K * np.exp(-r * T) * ss.norm.cdf(d2(S0, K, r, sigma, T))

# analytic solution for geometric brownian montion
def St_geom(sigma, S0, r, BT, T):
    S_t = S0*np.exp(sigma*BT+(r-sigma**2/2)*T)
    return S_t


#A Heston simulation-Milstein
def HestonMilstein(nu, rho, kappa, sigmasquare0, theta, 
                   T, m, K, cc=0, get_cc=False,
                   antithetic_v=False):
    """
    m: the steps of every simulation
    """
    W = np.random.normal(0, 1, m+1)
    Z = np.random.normal(0, 1, m+1)
    sigmasquare = np.zeros([m+1])
    integral = np.zeros([m+1])
    asset = np.zeros([m+1])
    integralstoch = np.zeros([m+1])
    integralstoch2 = np.zeros([m+1])
    brownie = np.zeros([m+1])
    sigmasquare[0] = sigmasquare0
    sums = np.zeros([m+1])
    asset[0] = 100
    init_price = 100
    integral_s = 0
    kc = (sigmasquare0)*(np.exp(T*nu**2)-1)/(nu**2)
    for i in range(0, m):
        sums[i] = -0.5*integral[i]+integralstoch[i]+integralstoch2[i]  
        asset[i] = asset[0]*np.exp(sums[i])
        sigmasquare[i+1] = sigmasquare[i] + kappa*(theta-sigmasquare[i])/m +\
                           nu*np.sqrt(sigmasquare[i])*W[i]*np.sqrt(T/m) +\
                           ((nu**2)/4)*(W[i]**2-1)/m
        if sigmasquare[i+1]<0:
            sigmasquare[i+1] = 0
        integral[i+1] = integral[i]+(T/m)*(sigmasquare[i])
        integralstoch[i+1] = integralstoch[i]+rho*np.sqrt(T/m)*np.sqrt(sigmasquare[i])*W[i]
        integralstoch2[i+1] = integralstoch2[i]+np.sqrt(1-rho**2)*np.sqrt(T/m)*np.sqrt(sigmasquare[i])*Z[i]
    
    sums[m] = -0.5*integral[m] + integralstoch[m] + integralstoch2[m]
    asset[m] = asset[0]*np.exp(sums[m])
    payoff = np.maximum(asset[m]-K, 0)+ cc*(asset[m] - init_price)
    if antithetic_v:
        W = -W
        Z = -Z
        sigmasquare = np.zeros([m+1])
        integral = np.zeros([m+1])
        asset = np.zeros([m+1])
        integralstoch = np.zeros([m+1])
        integralstoch2 = np.zeros([m+1])
        brownie = np.zeros([m+1])
        sigmasquare[0] = sigmasquare0
        sums = np.zeros([m+1])
        asset[0] = 100
        init_price = 100
        integral_s = 0
        kc = (sigmasquare0)*(np.exp(T*nu**2)-1)/(nu**2)
        for i in range(0, m):
            sums[i] = -0.5*integral[i]+integralstoch[i]+integralstoch2[i]  
            asset[i] = asset[0]*np.exp(sums[i])
            sigmasquare[i+1] = sigmasquare[i] + kappa*(theta-sigmasquare[i])/m +\
                               nu*np.sqrt(sigmasquare[i])*W[i]*np.sqrt(T/m) +\
                               ((nu**2)/4)*(W[i]**2-1)/m
            if sigmasquare[i+1]<0:
                sigmasquare[i+1] = 0
            integral[i+1] = integral[i]+(T/m)*(sigmasquare[i])
            integralstoch[i+1] = integralstoch[i]+rho*np.sqrt(T/m)*np.sqrt(sigmasquare[i])*W[i]
            integralstoch2[i+1] = integralstoch2[i]+np.sqrt(1-rho**2)*np.sqrt(T/m)*np.sqrt(sigmasquare[i])*Z[i]
        sums[m] = -0.5*integral[m] + integralstoch[m] + integralstoch2[m]
        asset[m] = asset[0]*np.exp(sums[m])
        payoff1 = np.maximum(asset[m]-K, 0)+ cc*(asset[m] - init_price)
    if get_cc:
        return payoff, asset[m]
    if antithetic_v:
        return (payoff + payoff1)/2
    return payoff


# get correlation between control variable and target variable with model.
def HestonMonteCarloMilstein_corre(nu, rho, kappa, sigmasquare0, theta, T, sim_n, step_n, K):
    """
    m: the number of steps
    n: the number of simulations
    """
    S0 = 100
    payoff = np.zeros([sim_n])
    S_prems = np.zeros([sim_n])
    for i in range(0, sim_n):
        payoff_i, S_prems_i = HestonMilstein(nu, rho, kappa, sigmasquare0,
                          theta, T, step_n, K, cc=0, get_cc=True)
        payoff[i] = payoff_i
        S_prems[i] = S_prems_i
    cc_1 = corr_c(S_prems, payoff, S0)
    return cc_1

# Heston Monte Carlo (normal Milstein)
def HestonMonteCarloMilstein(nu, rho, kappa, sigmasquare0, theta, 
                             T, sim_n, step_n, K, cc=0):
    """
    m: the number of steps
    n: the number of simulations
    """
    payoff = np.zeros([sim_n])
    for i in range(0, sim_n):
           payoff[i] = HestonMilstein(nu, rho, kappa, sigmasquare0,
                       theta, T, step_n, K, cc)
    mean = np.mean(payoff)
    variance = np.var(payoff)
    return mean, variance   

def HestonMonteCarloMilstein_anti(nu, rho, kappa, sigmasquare0, theta,
                                  T, sim_n, step_n, K):
    """
    m: the number of steps
    n: the number of simulations
    """
    payoff = np.zeros([sim_n])
    for i in range(0, sim_n):
           payoff[i] = HestonMilstein(nu, rho, kappa, sigmasquare0,
                       theta, T, step_n, K, 0, get_cc=False,
                       antithetic_v=True)
    mean = np.mean(payoff)
    variance = np.var(payoff)
    return mean, variance  


# get correlation parameter C:
def corr_c(S0_prem, CS, S0):
    S0_prem = np.array(S0_prem)
    CS = np.array(CS)
    cc = -np.sum((CS - np.mean(CS))*(S0_prem - S0))/(np.sum((S0_prem - S0)**2))
    return cc



# Heston price conditional MC, 
# Option price Heston with rho neq 0
def CMChestonprice(nu, rho, kappa, sigsquare,
                  theta, T, m, n, K, cc=0, 
                  get_c=False, 
                  antithetic_v=False):
    """ 
    contiditional Monte Carlo for Heston model, here we assume the interest rate r=0
    and the initial asset price is 100 that is the expecation of S0_prem.
    m: the the times of simulation
    n: the step size
    K: the strik price
    cc: the correlation bwtween the control variable and target variable.if cc is not 0, 
        it means you will use control variable method. If cc is 0, 
        you will not use control variable.
    get_c: if it is True, the return will be correlation bwtween
           the control variable and target variable.
    antithetic_v: if it is True, it means you will use antithetic method.
    """
    interest_r = 0
    init_price = 100
    S0_prems = np.zeros([m])
    mcheston = np.zeros([m])
    if antithetic_v:
        mcheston1 =np.zeros([m])
    for i in range(0, m):  
            W=np.random.normal(0, 1, n)
            sigmasquare=np.zeros([n+1])
            integral=0
            brownie=np.zeros([n+1])                                                                                                                     
            sigmasquare[0]=sigsquare
            integralstoch=0
            for l in range(0, n):
                brownie[l+1] = brownie[l]+W[l]*np.sqrt(T/n)
                sigmasquare[l+1] = sigmasquare[l]+kappa*(theta-sigmasquare[l])*(T/n)+ \
                                   nu*np.sqrt(sigmasquare[l])*W[l]*np.sqrt(T/n)+ \
                                   ((nu**2)/4)*(W[l]**2-1)*T/n
                if sigmasquare[l+1] < 0:
                    sigmasquare[l+1] =- sigmasquare[l+1]
                integral = integral+(T/n)*(sigmasquare[l])
                integralstoch = integralstoch+(np.sqrt(sigmasquare[l]))*np.sqrt(T/n)*W[l]
            S0_prem = init_price*np.exp(rho*integralstoch-0.5*integral*(rho**2))
            sig = np.sqrt((1-(rho**2))*integral/T)
            mcheston[i] = BlackScholes(S0_prem, K, interest_r, sig , T) + cc*(S0_prem-init_price)
            S0_prems[i] = S0_prem
            # ----------------------------------Antithetic-Variable-------------------------------------st
            if antithetic_v:
                W = - W
                sigmasquare = np.zeros([n+1])
                integral = 0
                brownie = np.zeros([n+1])                                                                                                                     
                sigmasquare[0] = sigsquare
                integralstoch = 0
                for l in range (0, n):
                    brownie[l+1] = brownie[l]+W[l]*np.sqrt(T/n)
                    sigmasquare[l+1] = sigmasquare[l]+kappa*(theta-sigmasquare[l])*(T/n)+ \
                                       nu*np.sqrt(sigmasquare[l])*W[l]*np.sqrt(T/n)+ \
                                       ((nu**2)/4)*(W[l]**2-1)*T/n
                    if sigmasquare[l+1] <0:
                        sigmasquare[l+1] =- sigmasquare[l+1]
                    integral = integral+(T/n)*(sigmasquare[l])
                    integralstoch = integralstoch+(np.sqrt(sigmasquare[l]))*np.sqrt(T/n)*W[l]
                S0_prem = init_price*np.exp(rho*integralstoch-0.5*integral*(rho**2))
                sig = np.sqrt((1-(rho**2))*integral/T)
                mcheston1[i] = BlackScholes(S0_prem, K, interest_r, sig , T) + cc*(S0_prem-init_price)
            # ----------------------------------Antithetic-Variable-------------------------------------ed
    if antithetic_v:
        mcheston = (mcheston + mcheston1)/2
    if get_c:
        return corr_c(S0_prems, mcheston, init_price)
    mean = np.mean(mcheston)
    var = np.var(mcheston)
    return mean, var

# 0.0 crude MonteCarloMilstein + control variate

In [110]:
sim_n = 1000
cc1 = HestonMonteCarloMilstein_corre(mu, rho, kappa, 
      sigsquare0, theta, T, sim_n, step_n, K)
print(cc1)
sim_n = 1000
p1, pv1 = HestonMonteCarloMilstein(mu, rho, kappa, 
          sigsquare0, theta, T, sim_n, step_n, K, cc=cc1)
print(p1, pv1)

-0.5173242107672855
9.17268281142175 52.853727737128935


In [139]:
n1 = 50
sim_n = 2000
mu = 0.3
rho = -0.5
kappa = 1
sigsquare0 = 0.04
theta = 0.09
T = 1
K = 100
sim_n = 1000
step_n = 1000
payoffs_MCM_CV = np.zeros([n1])
for i in tqdm(range(n1)):
    p_i, pv_i = HestonMonteCarloMilstein(mu, rho, kappa, 
          sigsquare0, theta, T, sim_n, step_n, K, cc=cc1)
    payoffs_MCM_CV[i] = p_i
print(np.mean(payoffs_MCM_CV), np.std(payoffs_MCM_CV))

100%|██████████| 50/50 [13:11<00:00, 15.83s/it]

9.238349583489706 0.2513413413497287


In [145]:
p1_anti, pv1_anti = HestonMonteCarloMilstein_anti(mu, rho, kappa, sigsquare0, theta,
                    T, sim_n, step_n, K)
print(p1_anti, pv1_anti)

8.955262091391408 54.18144808115544


In [140]:
n2 = int(n1/1)
payoffs_MCM_ANTI = np.zeros([n2])
for i in tqdm(range(n2)):
    p1_anti, pv1_anti = HestonMonteCarloMilstein_anti(mu, rho, kappa, sigsquare0, theta,
                                  T, sim_n, step_n, K)
    payoffs_MCM_ANTI[i] = p1_anti
print(np.mean(payoffs_MCM_ANTI), np.std(payoffs_MCM_ANTI))

100%|██████████| 50/50 [26:16<00:00, 31.52s/it]

9.211333250276503 0.23844546777206146


In [ ]:
    integral = 0
    integralstoch = 0
    delta_t = T/step_n
    for i in range(0, step_n):
        integral = integral + delta_t * sigmasquare
        integralstoch = integralstoch + np.sqrt(sigmasquare) * np.sqrt(delta_t) * W[i]
        # update sigmasquare
        sigmasquare = sigmasquare + kappa * (theta - sigmasquare) * delta_t + \
                      mu * np.sqrt(sigmasquare) * W[i] + \
                      ((mu ** 2) / 4) * (W[i] ** 2 - delta_t)
        if sigmasquare < 0:
            sigmasquare = -sigmasquare
    S0_prem = init_price * np.exp(rho * integralstoch - 0.5 * integral * (rho ** 2))
    sig = np.sqrt((1 - rho ** 2) * integral / T)
    payoff = BlackScholes(S0_prem, K, r, sig, T)

# 1 Simulate once 

# 1.1 Conditional Monte Carlo

In [147]:
import time
# assume the r=0
mu = 0.3
rho = -0.5
kappa = 1
sigsquare0 = 0.04
theta = 0.09
T = 1
K = 100
sim_n = 1000
step_n = 1000
t1 = time.time()
print(HestonMonteCarloMilstein(mu, rho, kappa, sigsquare0, theta, T, sim_n, step_n, K))
t2 = time.time()
print(t2-t1)
print(CMChestonprice(mu, rho, kappa, sigsquare0, theta, T, sim_n, step_n, K))
t3 = time.time()
print(t3-t2)

(9.02946960297695, 189.7957493128921)
16.87398672103882
(9.437835733630926, 20.30543112273086)
13.054864168167114


# 1.2 Conditional Monte Carlo +  Control Variable 

In [3]:
# estimate parameter correlation C
cc = CMChestonprice(mu, rho, kappa, sigsquare0, theta, T, sim_n, step_n, K,
                cc=0,
                get_c=True, 
                antithetic_v=False)
cc

100%|██████████| 1000/1000 [00:11<00:00, 87.17it/s]


-0.36447321924984266

In [4]:
t4 = time.time()
sim_n = 1000
p_mean, p_var = CMChestonprice(mu, rho, kappa, sigsquare0, theta, T, sim_n, step_n, K,
                cc=cc,
                get_c=False, 
                antithetic_v=False)
t5 = time.time()
print(p_mean, p_var)
print(t5-t4)

100%|██████████| 1000/1000 [00:11<00:00, 86.77it/s]

9.292436246125602 2.137443361086298
11.528409004211426


# 1.3 Conditional Monte Carlo + Antithetic Variable

In [5]:
sim_n = 500
t6 = time.time()
p_mean, p_var = CMChestonprice(mu, rho, kappa, sigsquare0, theta, T, sim_n, step_n, K,
                cc=0,
                get_c=False, 
                antithetic_v=True)
t7 = time.time()
print(p_mean, p_var)
print(t7-t6)

100%|██████████| 500/500 [00:11<00:00, 43.51it/s]

9.197018845703523 0.300882524661544
11.495227098464966


# 1.4 Conditional Monte Carlo + Control Variable + Antithetic Variable

In [8]:
sim_n = 500
t8 = time.time()
p_mean, p_var = CMChestonprice(mu, rho, kappa, sigsquare0, theta, T, sim_n, step_n, K,
                cc=cc,
                get_c=False, 
                antithetic_v=True)
t9 = time.time()
print(p_mean, p_var, t9-t8)

100%|██████████| 500/500 [00:11<00:00, 42.69it/s]

9.18650325451601 1.2158997590280818 11.716490983963013


# 5 results

|method|price|variance|times of  simulation|time cost(s)|
|--:|:--:|:--:|:--:|--:|
|Normal Milstein |8.8978|181.9831|1000|18.9230|
|Conditional Monte Carlo|9.5694|20.1060|1000|11.5447|
|Conditional Monte Carlo +  Control Variable |9.2924|2.1374|1000|11.5284|
|Conditional Monte Carlo + Antithetic Variable|9.1970|0.3009|500|11.4952|
|Conditional Monte Carlo + Control Variable + Antithetic Variable|9.1865|1.2159|500|11.7165|

# 2   simulate 50 times

# 2.1 Conditional Monte Carlo

In [31]:
sim_n = 1000
sim_num = 50
payoffs = np.zeros([sim_num])
payoffs_CMC = np.zeros([sim_num])
for i in tqdm(range(sim_num)):
    mean, var = HestonMonteCarloMilstein(mu, rho, kappa, sigsquare0, theta, T, sim_n, step_n, K)
    mean_CMC, var_CMC = CMChestonprice(mu, rho, kappa, sigsquare0, theta, T, sim_n, step_n, K)
    payoffs[i] = mean
    payoffs_CMC[i] = mean_CMC
print('Milstein, mean: {}, std {}'.format(np.mean(payoffs), np.std(payoffs)))
print('CMC, mean: {}, std {}'.format(np.mean(payoffs_CMC), np.std(payoffs_CMC)))

100%|██████████| 50/50 [23:11<00:00, 27.84s/it]

Milstein, mean: 9.27227211211279, std 0.4906567341879107
CMC, mean: 9.190927407358208, std 0.11249439804113703


# 2.2 Conditional Monte Carlo +  Control Variable 

In [35]:
sim_n = 1000
payoffs_CMC_CV = np.zeros([sim_num])
for i in tqdm(range(sim_num)):
    mean_CMC_CV, var_CMC_CV = CMChestonprice(mu, rho, kappa, sigsquare0, theta, T, sim_n, step_n, K,
                    cc=cc,
                    get_c=False, 
                    antithetic_v=False)
    payoffs_CMC_CV[i] = mean_CMC_CV
print('Milstein, mean: {}, std {}'.format(np.mean(payoffs_CMC_CV), np.std(payoffs_CMC_CV)))

100%|██████████| 50/50 [09:52<00:00, 11.85s/it]

Milstein, mean: 9.232755680153776, std 0.04684908848425236


# 2.3 Conditional Monte Carlo + Antithetic Variable

In [33]:
payoffs_CMC_ANTI = np.zeros([sim_num])
sim_n = 500
for i in tqdm(range(sim_num)):
    mean_CMC_ANTI, var_CMC_ANTI = CMChestonprice(mu, rho, kappa, sigsquare0, theta, T, sim_n, step_n, K,
                    cc=0,
                    get_c=False, 
                    antithetic_v=True)
    payoffs_CMC_ANTI[i] = mean_CMC_ANTI
print('Milstein, mean: {}, std {}'.format(np.mean(payoffs_CMC_ANTI), np.std(payoffs_CMC_ANTI)))

100%|██████████| 50/50 [09:01<00:00, 10.84s/it]

Milstein, mean: 9.22464297385613, std 0.025364488094739488


# 2.4 Conditional Monte Carlo + Control Variable + Antithetic Variable

In [30]:
payoffs_CMC_ALL = np.zeros([sim_num])
sim_n = 500
for i in tqdm(range(sim_num)):
    mean_CMC_ALL, var_CMC_ALL = CMChestonprice(mu, rho, kappa, sigsquare0, theta, T, sim_n, step_n, K,
                    cc=cc,
                    get_c=False, 
                    antithetic_v=True)
    payoffs_CMC_ALL[i] = mean_CMC_ALL
print('Milstein, mean: {}, std {}'.format(np.mean(payoffs_CMC_ALL), np.std(payoffs_CMC_ALL)))

100%|██████████| 50/50 [08:55<00:00, 10.72s/it]

Milstein, mean: 9.243498655738982, std 0.04928711117724456


# 2.5 Results

|method|price|variance|times of simulation|times of Monte Carlo|time cost (min)|
|--:|:--:|:--:|:--:|:--:|:--:|
|Normal Milstein |9.2723|0.4907|1000|50|11|
|Conditional Monte Carlo|9.1909|0.1125|1000|50|11|
|Conditional Monte Carlo +  Control Variable |9.2327|0.0468|1000|50|10|
|Conditional Monte Carlo + Antithetic Variable|9.2246|0.0254|500|50|9|
|Conditional Monte Carlo + Control Variable + Antithetic Variable|9.2435|0.0493|500|50|9|